<a href="https://colab.research.google.com/github/WonJunPark/Super_resolution/blob/master/SR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
import numpy as np

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()



train_images_gt = train_images
test_images_gt = test_images

#X->x->X'
im = []
for i in range(train_images.shape[0]):
  tmp_org = train_images[i]
  tmp = Image.fromarray(tmp_org)  
  tmp = tmp.resize((16, 16))#x
  tmp = tmp.resize((32, 32))#X'
  tmp = np.asarray(tmp)
  
#augmentation
#  tmp_flip_ud = np.flipud(tmp)
#  tmp_flip_lr = np.fliplr(tmp)
  im.append(np.expand_dims(tmp, axis=0))


train_images = np.concatenate(im, axis=0)/255.0
train_images_gt = train_images_gt/255.0


#X->x->X' SR!!
im = []
for i in range(test_images.shape[0]):

  tmp = Image.fromarray(test_images[i])
  tmp = tmp.resize((16, 16))#x
  tmp = tmp.resize((32, 32))#X'
  tmp = np.asarray(tmp)
  im.append(np.expand_dims(tmp, axis=0))
  
 

test_images = np.concatenate(im, axis=0)
test_images = test_images/255.0
test_images_gt = test_images_gt/255.0

print(train_images.shape, test_images.shape)


### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image.


In [ ]:
print(train_images_gt.shape)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    
    plt.imshow(train_images_gt[i], cmap=plt.cm.binary)
plt.show()


plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    
    plt.imshow(train_images[i], cmap=plt.cm.binary)
plt.show()

In [ ]:
conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')
conv2 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')
conv3 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')
conv4 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same')
conv5 = tf.keras.layers.Conv2D(3, 3, padding = 'same')


inputs = tf.keras.layers.Input(shape= [32,32,3])

tmp = conv1(inputs)
tmp = conv2(tmp)
tmp = conv3(tmp)
tmp = conv4(tmp)
tmp = conv5(tmp) + inputs

model = tf.keras.Model(inputs = inputs, outputs = tmp)


Let's display the architecture of our model so far.

In [ ]:
model.summary()
tf.keras.utils.plot_model(model, show_shapes = True)

### Compile and train the model

In [ ]:
model.compile(optimizer='adam',
              loss='mean_absolute_error')


model.fit(train_images, train_images_gt, epochs=10, 
                    validation_data=(test_images, test_images_gt))


### Evaluate the model

In [ ]:
prediction = model.predict(test_images)
print(prediction.shape)

prediction = np.maximum(0, np.minimum(prediction, 1))#guess what?

plt.figure(figsize=(10,10))
for i in range(8):
    plt.subplot(4,4,2*i+1)
    #plt.xticks([])
    #plt.yticks([])
    #plt.grid(False)
    plt.imshow(test_images[i], cmap=plt.cm.binary)

    plt.subplot(4,4,2*i+2)
    #plt.xticks([])
    #plt.yticks([])
    #plt.grid(False)
    plt.imshow(prediction[i], cmap=plt.cm.binary)

plt.show()


In [ ]:
print(test_acc)

Our simple CNN has achieved a test accuracy of over 70%. Not bad for a few lines of code! For another CNN style, see an example using the Keras subclassing API and a `tf.GradientTape` [here](https://www.tensorflow.org/tutorials/quickstart/advanced).